In [8]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data}, grad={self.grad})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [10]:
import random
class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    print("W:", self.w)
    self.b = Value(random.uniform(-1,1))
    print("b:", self.b)
  
  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [11]:
n = MLP(3, [4, 4, 1])

W: [Value(data=-0.34108091502837, grad=0.0), Value(data=0.4875444774924518, grad=0.0), Value(data=-0.5786687891714137, grad=0.0)]
b: Value(data=0.42066411352123567, grad=0.0)
W: [Value(data=-0.7406084700295616, grad=0.0), Value(data=-0.5419469493214886, grad=0.0), Value(data=-0.7157868250555495, grad=0.0)]
b: Value(data=-0.06842417572242421, grad=0.0)
W: [Value(data=-0.4315505068114813, grad=0.0), Value(data=-0.3888294993216346, grad=0.0), Value(data=-0.7260616380268912, grad=0.0)]
b: Value(data=0.3917250492343052, grad=0.0)
W: [Value(data=0.8544519713956533, grad=0.0), Value(data=-0.24436625968891335, grad=0.0), Value(data=-0.29745436944045234, grad=0.0)]
b: Value(data=0.07861738679252617, grad=0.0)
W: [Value(data=0.8240668394757076, grad=0.0), Value(data=-0.7925535106179675, grad=0.0), Value(data=0.856718957818575, grad=0.0), Value(data=0.38940887530348656, grad=0.0)]
b: Value(data=0.2807627460701372, grad=0.0)
W: [Value(data=-0.8401491947426882, grad=0.0), Value(data=0.6005090770290

In [12]:
xs = [
    [2.5, 3.5, -0.5],  # Primera muestra
    [4.0, -1.0, 0.5],  # Segunda muestra
    [0.5, 1.5, 1.0],   # Tercera muestra
    [3.0, 2.0, -1.5]   # Cuarta muestra
]
ys = [1.0, -1.0, -1.0, 1.0]

In [13]:
for k in range(5):  # Incrementa las iteraciones para ver más cambios
  
    # forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
  
    # Imprimir los valores de predicciones y pérdida (loss)
    print(f"Iteración {k}")
    print("Predicciones:", [yp.data for yp in ypred])
    print("Loss:", loss.data)
  
    # backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
  
    # Imprimir gradientes antes de la actualización
    print("Gradientes:")
    for i, p in enumerate(n.parameters()):
        print(f"Parámetro {i}: {p.data}, Gradiente: {p.grad}")
  
    # update
    for p in n.parameters():
        p.data += -0.1 * p.grad
  
    # Imprimir valores actualizados
    print("Pesos y bias actualizados:")
    for i, p in enumerate(n.parameters()):
        print(f"Parámetro {i}: {p.data}")
    print("-" * 50)

Iteración 0
Predicciones: [-0.09551180109351012, 0.6594294380423391, -0.2762009023561765, 0.08659177780839312]
Loss: 5.312051880293905
Gradientes:
Parámetro 0: -0.34108091502837, Gradiente: 0.04461671411143314
Parámetro 1: 0.4875444774924518, Gradiente: -0.059958272280120406
Parámetro 2: -0.5786687891714137, Gradiente: -1.3604871154282852
Parámetro 3: 0.42066411352123567, Gradiente: -0.607066022426714
Parámetro 4: -0.7406084700295616, Gradiente: 0.09660030344263507
Parámetro 5: -0.5419469493214886, Gradiente: 0.031391757949843134
Parámetro 6: -0.7157868250555495, Gradiente: 0.017082571888043213
Parámetro 7: -0.06842417572242421, Gradiente: 0.04373490518853666
Parámetro 8: -0.4315505068114813, Gradiente: -1.4462483586054862
Parámetro 9: -0.3888294993216346, Gradiente: -0.7648600096222877
Parámetro 10: -0.7260616380268912, Gradiente: 0.562176409487217
Parámetro 11: 0.3917250492343052, Gradiente: -0.41736488688980566
Parámetro 12: 0.8544519713956533, Gradiente: -0.38189231845456884
Paráme